<a href="https://colab.research.google.com/github/darinkist/Enhancing_LLMs_with_Search_APIs/blob/main/MediumArticle_Enhancing_LLMs_with_Search_APIs.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -q exa_py tavily-python langchain langchainhub langchain-openai llama-index llama-index-llms-langchain

In [2]:
import requests
from tavily import TavilyClient
from langchain.tools import tool
import json

from langchain import hub
from langchain.agents import AgentExecutor, create_openai_tools_agent
from langchain_openai import ChatOpenAI

from langchain.agents.format_scratchpad.openai_tools import (
    format_to_openai_tool_messages,
)
from langchain.agents.output_parsers.openai_tools import OpenAIToolsAgentOutputParser
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain.agents.format_scratchpad.openai_tools import (
    format_to_openai_tool_messages,
)
from langchain.agents.output_parsers.openai_tools import OpenAIToolsAgentOutputParser
from llama_index.core.evaluation import RelevancyEvaluator

from langchain_core.prompts import ChatPromptTemplate, PromptTemplate

from langchain_core.output_parsers import StrOutputParser
import time
import matplotlib.pyplot as plt

In [3]:
# YOU.COM API
@tool
def you_com_api(query: str) -> str:
  """Use this tool to search for latest information on the internet."""
  headers = {"X-API-Key": "XXXX"}
  params = {"query": query, "num_web_results":3}
  results = requests.get(
        f"https://api.ydc-index.io/search",
        params=params,
        headers=headers,
  ).json()
  return json.dumps(results)


# Exa Search API
@tool
def exa_search_api(query: str) -> str:
  """Use this tool to search for latest information on the internet."""
  url = "https://api.exa.ai/search"

  payload = { "query": query, "contents": { "text": { "includeHtmlTags": False } }, "numResults": 3 }
  headers = {
      "accept": "application/json",
      "content-type": "application/json",
      "x-api-key": "XXX"
  }
  response = requests.post(url, json=payload, headers=headers)

  return response.text


# Tavily Search
@tool
def tavily_search_api(query: str) -> str:
  """Use this tool to search for latest information on the internet."""
  tavily = TavilyClient(api_key="tvly-XXX")
  result = tavily.search(query=query,
                        search_depth="advanced",
                        max_results=3,
                        include_answer = True,
                        include_raw_content=False)
  return json.dumps(result)


# Perplexity AI
@tool
def perplexity_ai_api(query: str) -> str:
  """Use this tool to search for latest information on the internet."""
  url = "https://api.perplexity.ai/chat/completions"

  payload = {
      "model": "sonar-medium-online",
      "messages": [
          {
              "role": "system",
              "content": "Be precise and concise."
          },
          {
              "role": "user",
              "content": query
          }
      ]
  }
  headers = {
      "accept": "application/json",
      "content-type": "application/json",
      "authorization": "Bearer pplx-XXX"
  }

  response = requests.post(url, json=payload, headers=headers)
  return response.text

In [4]:
# Prepare agent
# Following https://python.langchain.com/docs/modules/agents/agent_types/openai_tools
prompt = hub.pull("hwchase17/openai-tools-agent")


def ask_agent(query, tool):

  # Bind tools to agent
  llm = ChatOpenAI(openai_api_key="XXX",
                   model="gpt-4-0125-preview",
                   temperature=0)
  llm_agent = llm.bind_tools([tool])

  # Define proper promt that includes tools
  prompt = ChatPromptTemplate.from_messages(
      [
          (
              "system",
              """You are very powerful assistant, but don't know current events.
              Always try to list the sources your answer is based on.
              """,
          ),
          ("user", "{input}"),
          MessagesPlaceholder(variable_name="agent_scratchpad"),
      ]
  )

  # Assembling llm and parts to get an agent
  agent = (
      {
          "input": lambda x: x["input"],
          "agent_scratchpad": lambda x: format_to_openai_tool_messages(
              x["intermediate_steps"]
          ),
      }
      | prompt
      | llm_agent
      | OpenAIToolsAgentOutputParser()
  )
  agent_executor = AgentExecutor(agent=agent, tools=[tool], verbose=True)
  result = agent_executor.invoke({"input": query})
  return result["output"]

In [5]:
def eval_service(query:str, agent_answer: str) -> dict:
  output_parser = StrOutputParser()
  instruction = """
  The current year is 2024.

  # Task:
  - Assess whether the AI's response adequately addresses the user's query provided below.

  # Instructions:
  1. Carefully read the user's query.
  2. Thoroughly review the AI's response.
  3. Rate on a scale from 1 to 5 the level of detail in the answer.
  4. Rate on a scale from 1 to 5 how up-to-date the information in the answer is, considering the date of the sources if provided.
  5. Rate on a scale from 1 to 5 the overall quality of the answer.

  # Evaluation Criteria:
  - **Detail Level:** Assess how thoroughly the answer addresses the query, considering all relevant aspects.
  - **Up-to-date Information:** Evaluate the currency and relevance of the information provided in the answer, taking into account the date of sources if provided.
  - **Answer Quality:** Judge the overall quality of the response, including accuracy, coherence, and relevance.

  # Options:
  - Choose one of the following:
    - "YES": If the AI's response effectively addresses the user's query, providing relevant and accurate information.
    - "NO": If the AI's response does not sufficiently address the user's query or is unrelated.

  # Return Format:
  - Provide your answer in JSON format with the keys:
    - "ANSWER": "YES" or "NO"
    - "DETAIL_RATING": (1-5)
    - "UP_TO_DATE_RATING": (1-5)
    - "QUALITY_RATING": (1-5)
    - "REASONING": Explain your ratings for each criterion.

  # User's Query:
  {query}

  # AI's Response:
  {model_answer}
  """

  prompt = PromptTemplate(
      template=instruction,
      input_variables=["query", "model_answer"]
  )

  gpt3 = ChatOpenAI(temperature=0,
                    model="gpt-4",
                    openai_api_key="XXX")

  chain = prompt | gpt3 | output_parser
  eval = chain.invoke({"query": query, "model_answer":agent_answer})
  return json.loads(eval)

In [6]:
questions = [
    "What are the top marketing trends in 2024?",
    "What are some significant geopolitical events that have occurred in February 2024?",
    "Could you summarize the most recent developments in artificial intelligence research and breakthroughs?",
    "What is the latest unemployment rate in the United States as reported by the Bureau of Labor Statistics?"
    ]

tools = [you_com_api, exa_search_api, tavily_search_api, perplexity_ai_api]

In [ ]:
ratings = {}
for tool in tools:
    ratings[tool.get_name()] = {}
    for q in questions:
        try:
            # time.sleep(2)
            tmp_answer = ask_agent(q, tool)
            if tmp_answer is not None:
                ratings[tool.get_name()][q] = eval_service(q, tmp_answer)
        except Exception as e:
            print(f"An error occurred while processing {q} with {tool}: {e}")